# FP-growth算法来高效发现频繁项集

### FP树：用于编码数据集的有效方式

FP-growth算法将数据存储在一种称为FP树的紧凑数据结构中。

FP代表频繁模式（Frequent Pattern）。

一棵FP树看上去与计算机科学中的其他树结构类似，但是它通过链接（link）来连接相似元素，被连起来的元素项可以看成一个链表。

与搜索树不同的是，一个元素项可以在一棵FP树种出现多次。

FP树会存储项集的出现频率，而每个项集会以路径的方式存储在数中。 

树节点上给出集合中的单个元素及其在序列中的出现次数，路径会给出该序列的出现次数。

相似项之间的链接称为节点链接（node link），用于快速发现相似项的位置。

**下图给出了FP树的一个例子**


```
事务集
```

| 事务ID        | 事务中的元素项   | 
| --------   | -----  | 
|001	|r, z, h, j, p|
|002|	z, y, x, w, v, u, t, s|
|003	|z|
|004|	r, x, n, o, s|
|005	|y, r, x, z, q, t, p|
|006	|y, z, x, e, q, s, t, m|

```
生成的FP树为
```

![这里写图片描述](https://img-blog.csdnimg.cn/img_convert/fa5f320274c7d97ea51142743aceb287.png)

**对FP树的解读：**

图中，元素项z出现了5次，集合{r, z}出现了1次。

于是可以得出结论：z一定是自己本身或者和其他符号一起出现了4次。

集合{t, s, y, x, z}出现了2次，集合{t, r, y, x, z}出现了1次，z本身单独出现1次。

就像这样，FP树的解读方式是读取某个节点开始到根节点的路径。

路径上的元素构成一个频繁项集，开始节点的值表示这个项集的支持度。

根据图，我们可以快速读出项集{z}的支持度为5、项集{t, s, y, x, z}的支持度为2、项集{r, y, x, z}的支持度为1、项集{r, s, x}的支持度为1。

FP树中会多次出现相同的元素项，也是因为同一个元素项会存在于多条路径，构成多个频繁项集。

但是频繁项集的共享路径是会合并的，如图中的{t, s, y, x, z}和{t, r, y, x, z}

和之前一样，我们取一个最小阈值，出现次数低于最小阈值的元素项将被直接忽略。图中将最小支持度设为3，所以q和p没有在FP中出现。

**FP-growth算法的工作流程如下:**

首先构建FP树，然后利用它来挖掘频繁项集。

为构建FP树，需要对原始数据集扫描两遍。

第一遍对所有元素项的出现次数进行计数。

数据库的第一遍扫描用来统计出现的频率，而第二遍扫描中只考虑那些频繁元素。

**FP-growth算法发现频繁项集的基本过程如下：**

 - 构建FP树 
 - 从FP树中挖掘频繁项集 

**头指针表**

FP-growth算法还需要一个称为头指针表的数据结构，其实很简单，就是用来记录各个元素项的总出现次数的数组，再附带一个指针指向FP树中该元素项的第一个节点。这样每个元素项都构成一条单链表。

图示说明：



![这里写图片描述](https://img-blog.csdnimg.cn/img_convert/2628b6024f09d414cb430b692028facd.png)

这里使用Python字典作为数据结构，来保存头指针表。以元素项名称为键，保存出现的总次数和一个指向第一个相似元素项的指针。

第一次遍历数据集会获得每个元素项的出现频率，去掉不满足最小支持度的元素项，生成这个头指针表。

**元素项排序**

上文提到过，FP树会合并相同的频繁项集（或相同的部分）。

因此为判断两个项集的相似程度需要对项集中的元素进行排序（不过原因也不仅如此，还有其它好处）。

排序基于元素项的绝对出现频率（总的出现次数）来进行。

在第二次遍历数据集时，会读入每个项集（读取），去掉不满足最小支持度的元素项（过滤），然后对元素进行排序（重排序）。

**构建FP树**

在对事务记录过滤和排序之后，就可以构建FP树了。

从空集开始，将过滤和重排序后的频繁项集一次添加到树中。如果树中已存在现有元素，则增加现有元素的值；如果现有元素不存在，则向树添加一个分支。

对前两条事务进行添加的过程：

![这里写图片描述](https://img-blog.csdnimg.cn/img_convert/ccee345cca3b9e9130fec721d71681cf.png)

**实现流程**

输入：数据集、最小值尺度
输出：FP树、头指针表

 - 1、遍历数据集，统计各元素项出现次数，创建头指针表
 - 2、移除头指针表中不满足最小值尺度的元素项
 - 3、第二次遍历数据集，创建FP树。对每个数据集中的项集：
  - 3.1 初始化空FP树
  - 3.2 对每个项集进行过滤和重排序
  - 3.3 使用这个项集更新FP树，从FP树的根节点开始：
     - 3.3.1 如果当前项集的第一个元素项存在于FP树当前节点的子节点中，则更新这个子节点的计数值
     - 3.3.2 否则，创建新的子节点，更新头指针表
     - 3.3.3 对当前项集的其余元素项和当前元素项的对应子节点递归3.3的过程


**从一棵FP树种挖掘频繁项集**

从FP树中抽取频繁项集的三个基本步骤如下：

 - 从FP树中获得条件模式基； 
 - 利用条件模式基，构建一个条件FP树； 
 - 迭代重复步骤1步骤2，直到树包含一个元素项为止。

其中“条件模式基”是以所查找元素项为结尾的路径集合。每一条路径其实都是一条前缀路径（prefix path）。

简而言之，一条前缀路径是介于所查找元素项与树根节点之间的所有内容。

例如

![这里写图片描述](https://img-blog.csdnimg.cn/img_convert/0d239b4d7b15973a7612b0d1d71ba36b.png)

则每一个频繁元素项的条件模式基为：

| 频繁项  |  前缀路径 |
| ----| ----|
|z|	{}: 5|
|r|{x, s}: 1, {z, x, y}: 1, {z}: 1|
|x|	{z}: 3, {}: 1|
|y|	{z, x}: 3|
|s|	{z, x, y}: 2, {x}: 1|
|t|	{z, x, y, s}: 2, {z, x, y, r}: 1|

发现规律了吗，z存在于路径{z}中，因此前缀路径为空，另添加一项该路径中z节点的计数值5构成其条件模式基；

r存在于路径{r, z}、{r, y, x, z}、{r, s, x}中，分别获得前缀路径{z}、{y, x, z}、{s, x}，另添加对应路径中r节点的计数值（均为1）构成r的条件模式基；

以此类推。

**创建条件FP树**

对于每一个频繁项，都要创建一棵条件FP树。可以使用刚才发现的条件模式基作为输入数据，并通过相同的建树代码来构建这些树。

例如

>对于r，即以“{x, s}: 1, {z, x, y}: 1, {z}: 1”为输入，调用函数createTree()获得r的条件FP树；

>对于t，输入是对应的条件模式基“{z, x, y, s}: 2, {z, x, y, r}: 1”。

**递归查找频繁项集**
 
有了FP树和条件FP树，我们就可以在前两步的基础上递归得查找频繁项集。

递归的过程是这样的：
输入：我们有当前数据集的FP树（inTree，headerTable）

 - 1、初始化一个空列表preFix表示前缀
 - 2、初始化一个空列表freqItemList接收生成的频繁项集（作为输出）
 - 3、对headerTable中的每个元素basePat（按计数值由小到大），递归：
  - 3.1 记basePat + preFix为当前频繁项集newFreqSet
  - 3.2 将newFreqSet添加到freqItemList中
  - 3.3 计算t的条件FP树（myCondTree、myHead）
  - 3.4 当条件FP树不为空时，继续下一步；否则退出递归
  - 3.5 以myCondTree、myHead为新的输入，以newFreqSet为新的preFix，外加freqItemList，递归这个过程

**实现代码**

In [1]:
# FP树类
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue  #节点元素名称，在构造时初始化为给定值
        self.count = numOccur   # 出现次数，在构造时初始化为给定值
        self.nodeLink = None   # 指向下一个相似节点的指针，默认为None
        self.parent = parentNode   # 指向父节点的指针，在构造时初始化为给定值
        self.children = {}  # 指向子节点的字典，以子节点的元素名称为键，指向子节点的指针为值，初始化为空字典

    # 增加节点的出现次数值
    def inc(self, numOccur):
        self.count += numOccur

    # 输出节点和子节点的FP树结构
    def disp(self, ind=1):
        print(' ' * ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind + 1)



In [2]:
# =======================================================构建FP树==================================================


# 对不是第一个出现的节点，更新头指针块。就是添加到相似元素链表的尾部
def updateHeader(nodeToTest, targetNode):
    while (nodeToTest.nodeLink != None):
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

# 根据一个排序过滤后的频繁项更新FP树
def updateTree(items, inTree, headerTable, count):
    if items[0] in inTree.children:
        # 有该元素项时计数值+1
        inTree.children[items[0]].inc(count)
    else:
        # 没有这个元素项时创建一个新节点
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        # 更新头指针表或前一个相似元素项节点的指针指向新节点
        if headerTable[items[0]][1] == None:  # 如果是第一次出现，则在头指针表中增加对该节点的指向
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])

    if len(items) > 1:
        # 对剩下的元素项迭代调用updateTree函数
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)



# 主程序。创建FP树。dataSet为事务集，为一个字典，键为每个事物，值为该事物出现的次数。minSup为最低支持度
def createTree(dataSet, minSup=1):
    # 第一次遍历数据集，创建头指针表
    headerTable = {}
    for trans in dataSet:
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    # 移除不满足最小支持度的元素项
    keys = list(headerTable.keys()) # 因为字典要求在迭代中不能修改，所以转化为列表
    for k in keys:
        if headerTable[k] < minSup:
            del(headerTable[k])
    # 空元素集，返回空
    freqItemSet = set(headerTable.keys())
    if len(freqItemSet) == 0:
        return None, None
    # 增加一个数据项，用于存放指向相似元素项指针
    for k in headerTable:
        headerTable[k] = [headerTable[k], None]  # 每个键的值，第一个为个数，第二个为下一个节点的位置
    retTree = treeNode('Null Set', 1, None) # 根节点
    # 第二次遍历数据集，创建FP树
    for tranSet, count in dataSet.items():
        localD = {} # 记录频繁1项集的全局频率，用于排序
        for item in tranSet:
            if item in freqItemSet:   # 只考虑频繁项
                localD[item] = headerTable[item][0] # 注意这个[0]，因为之前加过一个数据项
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)] # 排序
            updateTree(orderedItems, retTree, headerTable, count) # 更新FP树
    return retTree, headerTable



In [3]:
# =================================================查找元素条件模式基===============================================

# 直接修改prefixPath的值，将当前节点leafNode添加到prefixPath的末尾，然后递归添加其父节点。
# prefixPath就是一条从treeNode（包括treeNode）到根节点（不包括根节点）的路径
def ascendTree(leafNode, prefixPath):
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)

# 为给定元素项生成一个条件模式基（前缀路径）。basePet表示输入的频繁项，treeNode为当前FP树中对应的第一个节点
# 函数返回值即为条件模式基condPats，用一个字典表示，键为前缀路径，值为计数值。
def findPrefixPath(basePat, treeNode):
    condPats = {}  # 存储条件模式基
    while treeNode != None:
        prefixPath = []  # 用于存储前缀路径
        ascendTree(treeNode, prefixPath)  # 生成前缀路径
        if len(prefixPath) > 1:
            condPats[frozenset(prefixPath[1:])] = treeNode.count  # 出现的数量就是当前叶子节点的数量
        treeNode = treeNode.nodeLink  # 遍历下一个相同元素
    return condPats



In [4]:
# =================================================递归查找频繁项集===============================================
# 根据事务集获取FP树和频繁项。
# 遍历频繁项，生成每个频繁项的条件FP树和条件FP树的频繁项
# 这样每个频繁项与他条件FP树的频繁项都构成了频繁项集

# inTree和headerTable是由createTree()函数生成的事务集的FP树。
# minSup表示最小支持度。
# preFix请传入一个空集合（set([])），将在函数中用于保存当前前缀。
# freqItemList请传入一个空列表（[]），将用来储存生成的频繁项集。
def mineTree(inTree, headerTable, minSup, preFix, freqItemList):
    # 对频繁项按出现的数量进行排序进行排序
    sorted_headerTable = sorted(headerTable.items(), key=lambda p: p[1][0])  #返回重新排序的列表。每个元素是一个元组，[（key,[num,treeNode],()）
    bigL = [v[0] for v in sorted_headerTable]  # 获取频繁项
    for basePat in bigL:
        newFreqSet = preFix.copy()  # 新的频繁项集
        newFreqSet.add(basePat)     # 当前前缀添加一个新元素
        freqItemList.append(newFreqSet)  # 所有的频繁项集列表
        condPattBases = findPrefixPath(basePat, headerTable[basePat][1])  # 获取条件模式基。就是basePat元素的所有前缀路径。它像一个新的事务集
        myCondTree, myHead = createTree(condPattBases, minSup)  # 创建条件FP树

        if myHead != None:
            # 用于测试
            print('conditional tree for:', newFreqSet)
            myCondTree.disp()
            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList)  # 递归直到不再有元素


In [5]:
# 生成数据集
def loadSimpDat():
    simpDat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat

In [6]:
# 将数据集转化为目标格式
def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

if __name__=='__main__':
    minSup =3
    simpDat = loadSimpDat()  # 加载数据集
    initSet = createInitSet(simpDat)  # 转化为符合格式的事务集
    myFPtree, myHeaderTab = createTree(initSet, minSup)  # 形成FP树
    # myFPtree.disp()  # 打印树

    freqItems = []  # 用于存储频繁项集
    mineTree(myFPtree, myHeaderTab, minSup, set([]), freqItems)  # 获取频繁项集
    print(freqItems)  # 打印频繁项集

conditional tree for: {'y'}
  Null Set   1
   x   3
    z   3
conditional tree for: {'y', 'z'}
  Null Set   1
   x   3
conditional tree for: {'t'}
  Null Set   1
   y   3
    x   3
     z   3
conditional tree for: {'x', 't'}
  Null Set   1
   y   3
conditional tree for: {'z', 't'}
  Null Set   1
   y   3
    x   3
conditional tree for: {'z', 'x', 't'}
  Null Set   1
   y   3
conditional tree for: {'s'}
  Null Set   1
   x   3
conditional tree for: {'x'}
  Null Set   1
   z   3
[{'r'}, {'y'}, {'y', 'x'}, {'y', 'z'}, {'y', 'x', 'z'}, {'t'}, {'y', 't'}, {'x', 't'}, {'y', 'x', 't'}, {'z', 't'}, {'y', 'z', 't'}, {'z', 'x', 't'}, {'y', 'z', 'x', 't'}, {'s'}, {'x', 's'}, {'x'}, {'x', 'z'}, {'z'}]


**FP-growth算法总结**

FP-growth算法是一种用于发现数据集中频繁模式的有效方法。

FP-growth算法利用Apriori原则，执行更快。Apriori算法产生候选项集，然后扫描数据集来检查它们是否频繁。

由于只对数据集扫描两次，因此FP-growth算法执行更快。

在FP-growth算法中，数据集存储在一个称为FP树的结构中。

FP树构建完成后，可以通过查找元素项的条件基及构建条件FP树来发现频繁项集。

该过程不断以更多元素作为条件重复进行，直到FP树只包含一个元素为止。

优缺点：

 - 优点：一般要快于Apriori。 
 - 缺点：实现比较困难，在某些数据集上性能会下降。 
 - 适用数据类型：离散型数据。